In [1]:
import numpy as np
import pandas as pd

In [15]:
lenses = pd.read_csv('lenses.csv')

In [16]:
from scipy.spatial.distance import squareform, pdist

In [17]:
from scipy.sparse.csgraph import minimum_spanning_tree

In [21]:
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt

In [77]:
import sys

In [88]:
from sklearn.metrics.pairwise import pairwise_distances

In [137]:
def gini(x):
    # (Warning: This is a concise implementation, but it is O(n**2)
    # in time and memory, where n = len(x).  *Don't* pass in huge
    # samples!)

    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad/np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g

In [399]:
def counting(arr):
    mySet = set(arr)
    myList = []
    myNewSet = []
    for i in mySet:
        myNewSet.append(i)
        
    for i in range(len(myNewSet)):
        myList.append(0)
        for j in arr:
            if (myNewSet[i]==j):
                myList[i] += 1
    return myList

In [250]:
def menorCluster(clusters):
    mySet = set(clusters)
    minimo = len(clusters)
    menor = len(clusters)
    for i in mySet:
        if((clusters.count(i)) < minimo):
            minimo = clusters.count(i)
            menor = i
            
    myList = []
    
    for j in range(len(clusters)):
        if(clusters[j]==menor):
            myList.append(j)
            
    return myList

In [402]:
def genieLinkage(data, limiar):
    
    clusters = {}
    row_index = 0
    col_index = 0
    array = []
    

    for n in range(data.shape[0]):
        array.append(n)
    
        
    clusters[0] = array.copy()

    #finding minimum value from the distance matrix
    #note that this loop will always return minimum value from bottom triangle of matrix
    for k in range(1, data.shape[0]):
        min_val = sys.maxsize
        
        if(gini(counting(clusters[k-1])) <= limiar):
            for i in range(0, data.shape[0]):
                for j in range(0, data.shape[1]):
                    if(data[i][j]<=min_val):
                        min_val = data[i][j]
                        row_index = i
                        col_index = j
        else:
            iList = menorCluster(clusters[k-1])
            for i in iList:
                for j in range(0, data.shape[0]):
                    if(j not in iList):
                        if(data[i][j]<=min_val):
                            min_val = data[i][j]
                            row_index = i
                            col_index = j

        
        #once we find the minimum value, we need to update the distance matrix
        #updating the matrix by calculating the new distances from the cluster to all points
        
        #for Single Linkage
        
#         for i in range(0,data.shape[0]):
#             if(i != col_index):
#                 #we calculate the distance of every data point from newly formed cluster and update the matrix.
#                 temp = min(data[i][col_index],data[row_index][i])
#                 #we update the matrix symmetrically as our distance matrix should always be symmetric
#                 data[row_index][i] = temp
#                 data[i][col_index] = temp

                   
        #set the rows and columns for the cluster with higher index i.e. the row index to infinity
        #Set input[row_index][for_all_i] = infinity
        #set input[for_all_i][row_index] = infinity
#         data[row_index][col_index] = sys.maxsize
#         data[col_index][row_index] = sys.maxsize
            
        #Manipulating the dictionary to keep track of cluster formation in each step
        #if k=0,then all datapoints are clusters
        array = clusters[k-1].copy()
        minimum = min(array[row_index],array[col_index])
        maximum = max(array[row_index],array[col_index])
        
        for n in range(len(array)):
            if(array[n]==maximum):
                array[n] = minimum
        clusters[k] = array.copy()
        
        for i in range(len(array)):
            for j in range(len(array)):
                if(array[i]==array[j]):
                    data[i][j] = sys.maxsize
                    
        if (k>(data.shape[0]*0.8)):
            return clusters[k]

In [403]:
def genie(dataset, limiar):
    
#     dataX = dataset.loc[:,dataset.columns[:-1]].values
#     dataY = dataset.loc[:,[dataset.columns[-1]]].values
    dataX = dataset.data
    dataY = dataset.target
    
    dataDistances = pairwise_distances(dataX,metric='euclidean')
    #making all the diagonal elements infinity 
    np.fill_diagonal(dataDistances,sys.maxsize)
    
#     dataMST = minimum_spanning_tree(dataDistances)
    
#     dataMST = dataMST.toarray().astype(float)
    
    Z = genieLinkage(dataDistances, limiar)

    return Z

In [ ]:
iris = lo
genie(lenses, 0.36)

In [349]:
from sklearn.datasets import load_iris
iris = load_iris()

In [404]:
genie(iris, 0.4)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 14,
 14,
 0,
 0,
 0,
 0,
 20,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 20,
 14,
 14,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 50,
 51,
 50,
 53,
 54,
 55,
 51,
 57,
 54,
 53,
 57,
 61,
 62,
 63,
 62,
 54,
 55,
 62,
 68,
 53,
 70,
 71,
 70,
 63,
 71,
 54,
 54,
 50,
 63,
 53,
 53,
 53,
 62,
 83,
 55,
 51,
 50,
 68,
 61,
 53,
 61,
 63,
 62,
 57,
 61,
 61,
 61,
 71,
 57,
 61,
 100,
 101,
 102,
 103,
 104,
 105,
 55,
 107,
 104,
 109,
 110,
 110,
 112,
 101,
 114,
 100,
 103,
 117,
 105,
 70,
 109,
 114,
 105,
 70,
 109,
 102,
 70,
 70,
 104,
 102,
 107,
 117,
 104,
 83,
 103,
 107,
 100,
 103,
 70,
 112,
 109,
 141,
 101,
 109,
 109,
 141,
 70,
 110,
 100,
 70]